In [26]:
import tensorflow as tf
import numpy as np
import openai
from openai import OpenAI

model = tf.keras.models.load_model('ucf101_subset_model.h5')


In [28]:
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

In [29]:

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, None, None, None   0         
                             , 3)                                
                                                                 
 time_distributed (TimeDist  (None, None, None, None   4049571   
 ributed)                    , 1280)                             
                                                                 
 dense (Dense)               (None, None, None, None   129381    
                             , 101)                              
                                                                 
 global_average_pooling3d (  (None, 101)               0         
 GlobalAveragePooling3D)                                         
                                                                 
Total params: 4178952 (15.94 MB)
Trainable params: 12938

In [30]:
class_names = [
    'ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam',
    'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress',
    'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats',
    'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth',
    'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen',
    'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics',
    'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 'HammerThrow',
    'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace',
    'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls',
    'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 'LongJump', 'Lunges',
    'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars',
    'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute',
    'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'PlayingViolin',
    'PoleVault', 'PommelHorse', 'PullUps', 'Punch', 'PushUps', 'Rafting',
    'RockClimbingIndoor', 'RopeClimbing', 'Rowing', 'SalsaSpin', 'ShavingBeard',
    'Shotput', 'SkateBoarding', 'Skiing', 'Skijet', 'SkyDiving', 'SoccerJuggling',
    'SoccerPenalty', 'StillRings', 'SumoWrestling', 'Surfing', 'Swing',
    'TableTennisShot', 'TaiChi', 'TennisSwing', 'ThrowDiscus', 'TrampolineJumping',
    'Typing', 'UnevenBars', 'VolleyballSpiking', 'WalkingWithDog', 'WallPushups',
    'WritingOnBoard', 'YoYo'
]


In [31]:
import tensorflow as tf
import cv2
import numpy as np

model = tf.keras.models.load_model('ucf101_subset_model.h5')

def load_video(file_path, num_frames=10, resize_shape=(224, 224)):
    cap = cv2.VideoCapture(file_path)
    frames = []

    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, resize_shape)
        frames.append(frame)

    cap.release()
    
    frames = np.array(frames)
    frames = np.expand_dims(frames, axis=0)  # 추가 배치 차원
    frames = frames / 255.0  # 정규화

    return frames

video_path = 'div.avi'
video_frames = load_video(video_path)

# 예측 수행
predictions = model.predict(video_frames)
probabilities = tf.nn.softmax(predictions, axis=-1)
predicted_class = np.argmax(probabilities, axis=-1)

# 예측 결과 출력
print(f'Predicted class index: {predicted_class}')
print(f'Predicted class name: {class_names[predicted_class[0]]}')
print(f'Probabilities: {probabilities.numpy()}')

1/1 [==============================] - 1s 1s/step
Predicted class index: [25]
Predicted class name: Diving
Probabilities: [[1.07147102e-09 3.16574962e-08 6.05890662e-08 1.90081177e-08
  1.61140449e-07 1.94340686e-08 5.61583180e-09 9.39981765e-05
  1.95116681e-05 1.20651031e-10 2.15166177e-10 2.07148886e-07
  1.61664762e-08 3.30183797e-12 4.33979935e-10 2.05558095e-08
  2.64128053e-09 7.93943489e-09 4.12558642e-04 6.71420752e-10
  2.17018624e-06 4.18109741e-10 1.70084373e-08 7.55724869e-08
  1.13782699e-08 9.83509839e-01 4.43505499e-09 1.41818447e-07
  4.41385710e-05 1.19006891e-05 4.13499443e-11 5.79241532e-06
  5.20200372e-09 8.63448896e-11 1.31288471e-06 2.30408539e-10
  1.63925307e-07 2.90622506e-06 1.24399949e-10 1.17815379e-02
  5.42821121e-10 1.16883356e-08 6.46549665e-07 4.54059347e-07
  5.69759422e-08 2.57200270e-07 2.19018398e-06 2.28821267e-11
  6.74517082e-07 3.37286991e-07 2.63162178e-06 4.77600452e-07
  1.81504767e-08 5.59187896e-09 1.78056536e-09 1.11900633e-07
  3.704502

In [32]:
# OpenAI 클라이언트 설정
client = OpenAI(
    api_key=OPENAI_API_KEY
)

# 질문 작성하기
query = f"""I will draw a picture of the predicted class name: {class_names[predicted_class[0]]}. Set up the character, composition, 
and background. And write a prompt for generating the picture. 
The more detailed, the better. Include all the necessary information for drawing. Make sure the predicted class name is well reflected."""

# 메시지 설정하기
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that helps provide prompt about drawing a picture."
    },
    {
        "role": "user",
        "content": query
    }
]

response = client.chat.completions.create(

    model="gpt-3.5-turbo",
    messages=messages
)

# 응답 받기
answer = response.choices[0].message.content
print(answer)


Prompt for Generating the Picture:

Title: "Ocean Grace"

Character Description:
- The main subject of the drawing is a graceful and skilled female diver in mid-dive, captured in a moment of elegance and precision.
- The diver should be portrayed wearing a sleek, form-fitting diving suit that shimmers in the sunlight.
- Her face is focused and serene, displaying a sense of determination and passion for her craft.
- She has long, flowing hair trailing behind her, adding a sense of movement to the scene.
- The diver's body is in a perfect arched position, showcasing her expertise and control in the water.

Composition:
- The diver is positioned in the center of the composition, occupying the foreground of the image.
- The surrounding water is depicted in shades of turquoise and deep blue, with sunlight filtering through the surface, creating dappled patterns on the seabed below.
- Include air bubbles rising from the diver's mouth and small, colorful fish darting around her, adding to the

In [33]:
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

seed = 42
torch.manual_seed(seed)

def generate_image(prompt, num_inference_steps=50, guidance_scale=7.5, size=(1024, 576)):
    image = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
    image = image.resize(size)
    return image

answer = class_names[predicted_class[0]]

# 이미지 생성
prompt = f"A scene of {answer}"
generated_image = generate_image(prompt)

# 이미지 저장
current_dir = os.getcwd()
output_image_path = os.path.join(current_dir, 'generated_image.png')
generated_image.save(output_image_path)
print(f'Image saved at {output_image_path}')


100%|██████████| 50/50 [00:04<00:00, 10.16it/s]


Image saved at c:\ucfdiffu\generated_image.png


In [ ]:
import torch

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid", torch_dtype=torch.float16, variant="fp16"
)

pipe.enable_model_cpu_offload()

# Load the conditioning image
image = "generated_image.png"
image = Image.open(image).convert("RGB")

generator = torch.manual_seed(42)

frames = pipe(image, decode_chunk_size=2, generator=generator, num_frames=10).frames[0]


export_to_video(frames, "generated.mp4", fps=7)
